# Scotch whisky selection design

In [1]:
import pandas as pd
import numpy as np

In [12]:
info = pd.read_csv("whiskies-attributes-prices.csv")
rt = pd.read_csv("whiskies-utilities.csv")

# Part 2: Solving the problem

In [13]:
rt['NoPurchase'] = 27
rt_arr = np.array(rt)

In [4]:
nProducts = 86
nCustomers = 100

In [15]:
def expected_rev(S):
    S2 = np.append(S, nProducts)
    
    choice_probabilities = {}
    for i in S2:
        choice_probabilities[i] = 0.0;
    
    for k in range(nCustomers):
        ind = np.argmax(rt_arr[k,S2])
        choice_probabilities[ S2[ind] ] += 1.0/nCustomers
        
    exp_profit = sum( [choice_probabilities[i] * info['Price'][i] for i in S])
        
    return exp_profit, choice_probabilities

In [6]:
from gurobipy import *

m = Model()

# Create the decision variables
x = m.addVars(nProducts, vtype = GRB.BINARY )
y = m.addVars(nCustomers, nProducts+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints:")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( rt_arr[k,j] * y[k,j] for j in range(nProducts+1)) >= rt_arr[k,i] * x[i] + rt_arr[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( rt_arr[k,j] * y[k,j] for j in range(nProducts+1)) >= rt_arr[k,nProducts] )



# Create the objective:
print("Creating objective:")
m.setObjective( quicksum(info['Price'][i] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()


S_optimal = [i for i in range(nProducts) if x[i].x > 0.5]
print("Optimal set of products: ", S_optimal)

optimal_profit = m.objval
print("Optimal profit: ",optimal_profit)

Using license file /Users/wjq/gurobi.lic
Academic license - for non-commercial use only - expires 2021-08-08
Creating constraints:
Creating objective:
Update completed
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17400 rows, 8786 columns and 339435 nonzeros
Model fingerprint: 0x2de495af
Variable types: 0 continuous, 8786 integer (8786 binary)
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [3e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 12492 rows and 6276 columns
Presolve time: 0.77s
Presolved: 4908 rows, 2510 columns, 55559 nonzeros
Variable types: 0 continuous, 2510 integer (2510 binary)
Found heuristic solution: objective 107.5200000

Root relaxation: objective 1.345310e+02, 654 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

In [26]:
# Verify that optimal profit is correct:
exp_profit_S_optimal, choice_probabilities_S_optimal = expected_rev( S_optimal )
print("Optimal profit (via expected_profit): ", exp_profit_S_optimal)


for i in choice_probabilities_S_optimal.keys():
    print(rt.columns[i] + ": " + str(choice_probabilities_S_optimal[i]) )

Optimal profit (via expected_profit):  130.35
Auchentoshan: 0.07
Balblair: 0.060000000000000005
Balmenach: 0.05
Balvenie: 0.05
Bowmore: 0.08
GlenDeveronMacduff: 0.07
Lagavulin: 0.24000000000000007
Laphroaig: 0.07
Miltonduff: 0.13999999999999999
RoyalLochnagar: 0.07
Tomatin: 0.02
NoPurchase: 0.08


# Part 3 Balancing the whisky selection

In [31]:
for i in choice_probabilities_S_optimal.keys():
    if i != 86:
        print(rt.columns[i], info.iloc[i,4])

Auchentoshan 0
Balblair 1
Balmenach 0
Balvenie 0
Bowmore 1
GlenDeveronMacduff 1
Lagavulin 4
Laphroaig 4
Miltonduff 0
RoyalLochnagar 0
Tomatin 0


# Part 4: Extending the formulation

In [37]:
for i in range(nProducts):
    if rt.columns[i] == 'Ardbeg':
        print('Ardbeg',i+1)
    elif rt.columns[i] == 'Bowmore':
        print('Bowmore', i+1)
    elif rt.columns[i] == 'Oban':
        print('Oban', i+1)
    elif rt.columns[i] == 'Talisker':
        print('Talisker', i+1)

Ardbeg 4
Bowmore 19
Oban 67
Talisker 78


In [39]:
info

,Distillery,Body,Sweetness,Smoky,Medicinal,Tobacco,Honey,Spicy,Winey,Nutty,Malty,Fruity,Floral,Price
0,Aberfeldy,2,2,2,0,0,2,1,2,2,2,2,2,45
1,Aberlour,3,3,1,0,0,4,3,2,2,3,3,2,86
2,AnCnoc,1,3,2,0,0,2,0,0,2,2,3,2,52
3,Ardbeg,4,1,4,4,0,0,2,0,1,2,1,0,60
4,Ardmore,2,2,2,0,0,1,1,1,2,3,1,1,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Tobermory,1,1,1,0,0,1,0,0,1,2,2,2,105
82,Tomatin,2,3,2,0,0,2,2,1,1,2,0,1,35
83,Tomintoul,0,3,1,0,0,2,2,1,1,2,1,2,90
84,Tormore,2,2,1,0,0,1,0,1,2,1,0,0,110
